# compute AGI based on annual fields 
# present-day vs future
# use AGIcrit, Tpref, O2thresh as calculated from monthly hist. clim.
# HERE: save numbers for plotting as netcdf

In [1]:
#get_ipython().system(u'jupyter nbconvert --to=python plot_AGI_toothfish_testing.ipynb')

In [2]:

import xarray as xr
import numpy as np
import sys, os
from pathlib import Path
import glob
import matplotlib.pyplot as plt


In [3]:
#-----
# based on python code provided by Anne
#-----

basepath="/pscratch/sd/c/cnissen/AGI_toothfish_project/" 
no_species=29 

# Init
species_names         = [None] * no_species # empty list
species_names_        = [None] * no_species # empty list
habitat_file_names    = [None] * no_species # empty list

habitat_files = Path(basepath + "all_prey_plus_toothfish/").glob('*_boolean.nc')  # updated habitat files, Nov 2022!!!!!!
#habitat_files = Path(basepath + "share/").glob('*_boolean.nc') 
# Note: I changed name of file of Antimora_rostrata to *boolean2.nc, so that it won't be included here
for ifile,file in enumerate(habitat_files):   
    habitat_file_names[ifile]    = str(file)
    # Get the species name from the full pathname with a space between
    species_names[ifile]  = '_'.join(os.path.basename(file).split('_')[:-2]).replace('_',' ') 
    if not species_names[ifile] in ['Galiteuthis glacialis','Mesonychoteuthis hamiltoni','Kondakovia longimana']:
        species_names[ifile] = species_names[ifile][8:] # get rid of "Default" or "Reviewed"
    if species_names[ifile][0].isspace():  # get rid of white space if there is any
        species_names[ifile] = species_names[ifile][1:]
    if species_names[ifile] in ['Chionobathyscus dewitti All Suitable Habitat']:
        species_names[ifile] = species_names[ifile][0:23]
    # Get the species name from the full pathname with _ in it
    species_names_[ifile] = species_names[ifile].replace(' ','_')  #'_'.join(os.path.basename(file).split('_')[:-2]) 

#print(habitat_file_names)
print(species_names)
print(len(species_names),'species')
print(species_names_)

    
species_names_no_toothfish = ['Trematomus loennbergii', 'Macrourus whitsoni', 'Galiteuthis glacialis',\
                              'Chionodraco hamatus', 'Anotopterus vorax', 'Chaenodraco wilsoni',\
                              'Kondakovia longimana', 'Cryodraco antarcticus', 'Muraenolepis microps',\
                              'Psychroteuthis glacialis', 'Gymnodraco acuticeps', 'Amblyraja georgiana',\
                              'Antarctomysis maxima', 'Lepidonotothen squamifrons', 'Mesonychoteuthis hamiltoni',\
                              'Pleuragramma antarctica', 'Euphausia superba', 'Notothenia coriiceps',\
                              'Trematomus lepidorhinus', 'Pagothenia borchgrevinki', 'Lycenchelys aratrirostris',\
                              'Neopagetopsis ionah', 'Chaenocephalus aceratus', 'Trematomus hansoni',\
                              'Chionobathyscus dewitti', 'Euphausia crystallorophias', 'Notothenia rossii',\
                              'Bathyraja maccaini']
print (len(species_names_no_toothfish))


['Dissostichus mawsoni', 'Trematomus loennbergii', 'Macrourus whitsoni', 'Galiteuthis glacialis', 'Chionodraco hamatus', 'Anotopterus vorax', 'Chaenodraco wilsoni', 'Kondakovia longimana', 'Cryodraco antarcticus', 'Muraenolepis microps', 'Psychroteuthis glacialis', 'Gymnodraco acuticeps', 'Amblyraja georgiana', 'Antarctomysis maxima', 'Lepidonotothen squamifrons', 'Mesonychoteuthis hamiltoni', 'Pleuragramma antarctica', 'Euphausia superba', 'Notothenia coriiceps', 'Trematomus lepidorhinus', 'Pagothenia borchgrevinki', 'Lycenchelys aratrirostris', 'Neopagetopsis ionah', 'Chaenocephalus aceratus', 'Trematomus hansoni', 'Chionobathyscus dewitti', 'Euphausia crystallorophias', 'Notothenia rossii', 'Bathyraja maccaini']
29 species
['Dissostichus_mawsoni', 'Trematomus_loennbergii', 'Macrourus_whitsoni', 'Galiteuthis_glacialis', 'Chionodraco_hamatus', 'Anotopterus_vorax', 'Chaenodraco_wilsoni', 'Kondakovia_longimana', 'Cryodraco_antarcticus', 'Muraenolepis_microps', 'Psychroteuthis_glacialis'

In [4]:
#----
# Species information needed for calculation AGI
#----

# define order of species as contained in depth_min, LWa etc.
species_list = ['Cryodraco antarcticus','Neopagetopsis ionah','Trematomus lepidorhinus',\
               'Trematomus hansoni','Bathyraja maccaini','Chaenocephalus aceratus',\
               'Notothenia coriiceps','Pleuragramma antarctica','Dissostichus mawsoni',\
                'Macrourus whitsoni',\
               'Lepidonotothen squamifrons','Notothenia rossii','Muraenolepis microps',\
               'Chaenodraco wilsoni','Chionobathyscus dewitti',\
               'Galiteuthis glacialis','Mesonychoteuthis hamiltoni','Kondakovia longimana',\
               'Amblyraja georgiana','Anotopterus vorax','Antarctomysis maxima',\
               'Chionodraco hamatus','Euphausia crystallorophias','Euphausia superba','Gymnodraco acuticeps',\
               'Lycenchelys aratrirostris','Pagothenia borchgrevinki','Psychroteuthis glacialis',\
               'Trematomus loennbergii']
print(len(species_list))
# excluded the following species for now: 
# 1) Antimora rostrata: don't have the habitat file (habitat outside of SO)

# NO MAP Bathyraja eatonii	?		15	1500

# Minimum depth of occurence (meter)
depth_min     = [None] * len(species_names) # Initialize
depth_min[0]  = 90   # 'Cryodraco antarcticus'
depth_min[1]  = 20   #'Neopagetopsis ionah'
depth_min[2]  = 272    # 'Trematomus lepidorhinus'
depth_min[3]  = 6    # 'Trematomus hansoni'
depth_min[4]  = 167  # 'Bathyraja maccaini'
depth_min[5]  = 0    # 'Chaenocephalus aceratus'
depth_min[6]  = 0    # 'Notothenia coriiceps'
depth_min[7]  = 0    # 'Pleuragramma antarctica'
depth_min[8]  = 0    # 'Dissostichus mawsoni' ## Assume a wide range for toothfish (max. overlap with prey) ##
depth_min[9]  = 400  # 'Macrourus whitsoni'
depth_min[10] = 10   # 'Lepidonotothen squamifrons'
depth_min[11] = 5    # 'Notothenia rossii'
depth_min[12] = 10   # 'Muraenolepis microps'
depth_min[13] = 200  # 'Chaenodraco wilsoni'
depth_min[14] = 500  # 'Chionobathyscus dewitti'
depth_min[15] = 200   #'Galiteuthis glacialis'
depth_min[16] = 200   # 'Mesonychoteuthis hamiltoni'
depth_min[17] = 500   # 'Kondakovia longimana'
depth_min[18] = 57    # Amblyraja georgiana
depth_min[19] = 358  # Anotopterus vorax
depth_min[20] = 220 # Antarctomysis maxima
depth_min[21] = 76 # Chionodraco hamatus
depth_min[22] = 0 # Euphausia crystallorophias
depth_min[23] = 0 # Euphausia superba
depth_min[24] = 66 # Gymnodraco acuticeps
depth_min[25] = 244 # Lycenchelys aratrirostris
depth_min[26] = 0 # Pagothenia borchgrevinki
depth_min[27] = 385 # Psychroteuthis glacialis
depth_min[28] = 65 # Trematomus loennbergii
#depth_min[29] = # Trematomus eulepidotus #EXCLUDED

# Maximum depth of occurence (meter)
depth_max     = [None] * len(species_names) # Initialize
depth_max[0]  = 600  # 'Cryodraco antarcticus'
depth_max[1]  = 900  #'Neopagetopsis ionah'
depth_max[2]  = 468  # 'Trematomus lepidorhinus'
depth_max[3]  = 549  # 'Trematomus hansoni'
depth_max[4]  = 500  # 'Bathyraja maccaini'
depth_max[5]  = 770  # 'Chaenocephalus aceratus'
depth_max[6]  = 550  # 'Notothenia coriiceps'
depth_max[7]  = 1000 # 'Pleuragramma antarctica'
depth_max[8]  = 2210   # 'Dissostichus mawsoni' ## Assume a wide range for toothfish (max. overlap with prey) ##
depth_max[9]  = 3185 # 'Macrourus whitsoni'
depth_max[10] = 900  # 'Lepidonotothen squamifrons'
depth_max[11] = 350 # 'Notothenia rossii'
depth_max[12] = 1600 # 'Muraenolepis microps'
depth_max[13] = 800  # 'Chaenodraco wilsoni'
depth_max[14] = 2000 # 'Chionobathyscus dewitti'
depth_max[15] = 2500   #'Galiteuthis glacialis'
depth_max[16] =  600  # 'Mesonychoteuthis hamiltoni'
depth_max[17] = 2000   # 'Kondakovia longimana'
depth_max[18] = 173    # Amblyraja georgiana
depth_max[19] = 1059 # Anotopterus vorax
depth_max[20] = 440 # Antarctomysis maxima
depth_max[21] = 271 # Chionodraco hamatus
depth_max[22] = 650 # Euphausia crystallorophias
depth_max[23] = 600 # Euphausia superba
depth_max[24] = 247 # Gymnodraco acuticeps
depth_max[25] = 376 # Lycenchelys aratrirostris
depth_max[26] = 30 # Pagothenia borchgrevinki
depth_max[27] = 610 # Psychroteuthis glacialis
depth_max[28] = 832 # Trematomus loennbergii
#depth_max[29] = # Trematomus eulepidotus EXCLUDED

#---
# NOTE Jan 2023: this info below is not needed anymore with the new AGI formulation!
#----
## LWa extracted 21.07.2022 from FishBase
#LWa     = [None] * len(species_names) # Initialize
#LWa[0]  = 0.0007  # 'Cryodraco antarcticus'
#LWa[1]  = 0.01863  #'Neopagetopsis ionah'
#LWa[2]  = 0.0042  # 'Trematomus eulepidotus'
#LWa[3]  = 0.0021  # 'Trematomus hansoni'
#LWa[4]  = 0.00477  # 'Bathyraja maccaini'
#LWa[5]  = 0.0006 # 'Chaenocephalus aceratus'
#LWa[6]  = 0.0132  # 'Notothenia coriiceps'
#LWa[7]  = 0.0019  # 'Pleuragramma antarctica'
#LWa[8]  = 0.0045   # 'Dissostichus mawsoni'
#LWa[9]  = 0.0135 # 'Macrourus whitsoni'
#LWa[10] = 0.0027  # 'Lepidonotothen squamifrons'
##LWa[11] = 0.001     # 'Antimora rostrata'
#LWa[11] = 0.0093 # 'Notothenia rossii'
#LWa[12] = 0.00437 # 'Muraenolepis microps'
#LWa[13] = 0.0005  # 'Chaenodraco wilsoni'
#LWa[14] = 0.0012 # 'Chionobathyscus dewitti'

## LWb extracted 21.07.2022 from FishBase
#LWb     = [None] * len(species_names) # Initialize
#LWb[0]  = 3.51  # 'Cryodraco antarcticus'
#LWb[1]  = 2.762  #'Neopagetopsis ionah'
#LWb[2]  = 3.32  # 'Trematomus eulepidotus'
#LWb[3]  = 3.52  # 'Trematomus hansoni'
#LWb[4]  = 3.162  # 'Bathyraja maccaini'
#LWb[5]  = 3.63  # 'Chaenocephalus aceratus'
#LWb[6]  = 3.09  # 'Notothenia coriiceps'
#LWb[7]  = 3.41  # 'Pleuragramma antarctica'
#LWb[8]  = 3.24   # 'Dissostichus mawsoni'
#LWb[9]  = 3.15 # 'Macrourus whitsoni'
#LWb[10] = 3.41  # 'Lepidonotothen squamifrons'
##LWb[11] = 3.52   # 'Antimora rostrata'
#LWb[11] = 3.07 # 'Notothenia rossii'
#LWb[12] = 3.11 # 'Muraenolepis microps'
#LWb[13] = 3.79  # 'Chaenodraco wilsoni'
#LWb[14] = 3.5 # 'Chionobathyscus dewitti'

## Linf extracted 21.07.2022 from FishBase
## * Linf estimated from Lmax using Froese and Binohlan (2000) Eq. (5).
#Linf     = [None] * len(species_names) # Initialize
#Linf[0]  = 50.66374052  # 'Cryodraco antarcticus'*
#Linf[1]  = 58.12886422  #'Neopagetopsis ionah'*
#Linf[2]  = 26.5  # 'Trematomus eulepidotus'
#Linf[3]  = 36.5  # 'Trematomus hansoni'
#Linf[4]  = 123.061517  # 'Bathyraja maccaini'*
#Linf[5]  = 70.4  # 'Chaenocephalus aceratus'
#Linf[6]  = 62  # 'Notothenia coriiceps'
#Linf[7]  = 25.1  # 'Pleuragramma antarctica'
#Linf[8]  = 183   # 'Dissostichus mawsoni'
#Linf[9]  = 92 # 'Macrourus whitsoni'
#Linf[10] = 56.7  # 'Lepidonotothen squamifrons'
##Linf[11] = 66   # 'Antimora rostrata'
#Linf[11] = 87 # 'Notothenia rossii'
#Linf[12] = 36.60305736 # 'Muraenolepis microps'*
#Linf[13] = 44.82252387  # 'Chaenodraco wilsoni'*
#Linf[14] = 62.21264207 # 'Chionobathyscus dewitti'*


29


In [5]:
#----
# get indices of depth levels of species
#----

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

file_mesh = '/pscratch/sd/c/cnissen/files_toothfish_project_AGI/Mesh_ancillary_information_v20220919.nc'
data_levels = xr.open_dataset(file_mesh)
levels      = data_levels['depth'].values # 88 levels
data_levels.close()

index_min_depth     = [None] * len(species_names) # Initialize
index_max_depth     = [None] * len(species_names) # Initialize

for i,name in enumerate(species_names):
    # Note Cara: order of entries in depth_min etc. is not alphabetical!!!!
    # here, I have adapted the index from "i" to "iii" to read in the correct depth ranges
    # for the current species
    iii = species_list.index(species_names[i]) # get index of current species as contained in depth_min etc.
    index_min_depth[i] = find_nearest(levels,depth_min[iii])
    index_max_depth[i] = find_nearest(levels,depth_max[iii])
    # now, index_min_depth should contain the correct values according to alphabetical order as looped over
    # NOTE: given how python is indexing, I think I then need to index over index_min_depth:index_max_depth+1
    # (was index_min_depth:index_max_depth before)
    

In [6]:
#-----
# calculate AGI from annual mean fields
#-----

# constants
d = 0.7 # n.d.; metabolic scaling coefficient
j1 = 4500 # K; "Anabolism activation energy divided by Boltzmann constant" (Clarke2021)
j2 = 8000 # K; "Catabolism activation energy divided by Boltzmann constant" (Clarke2021)
j_diff = j2-j1
# from Clarke2021: "Somatic (or biomass) growth can be expressed as the difference between anabolism and catabolism"

#---
# load AGIcrit, Tpref, O2thresh
#---

# Init
AGI_crit = [None] * no_species # empty list
o2thresh = [None] * no_species # empty list
TPref    = [None] * no_species # empty list

# AGIcrit 
for ii in range(0,len(species_names)):
    file = 'AGIcrit_'+species_names_[ii]+'_based_on_monthly_clim_1995_2014.nc'
    #print(file)
    ff     = xr.open_dataset(basepath+'new_thresholds/'+file)
    AGI_crit[ii]        = ff['AGIcrit'].values[0]
    ff.close()
    
# o2thresh  
for ii in range(0,len(species_names)):
    file = 'o2thresh_'+species_names_[ii]+'_based_on_monthly_clim_1995_2014.nc'
    ff     = xr.open_dataset(basepath+'new_thresholds/'+file)
    o2thresh[ii]        = ff['o2thresh'].values[0]
    ff.close()
    
# TPref 
for ii in range(0,len(species_names)):
    file = 'Tpref_'+species_names_[ii]+'_based_on_monthly_clim_1995_2014.nc'
    ff     = xr.open_dataset(basepath+'new_thresholds/'+file)
    TPref[ii]        = ff['Tpref'].values[0]
    ff.close()

#-----
# NOTE that order of species in LWa etc is not the same as in "species_names"
# account for that further down!!!!
#-----
# UPDATE Jan 2023: the info below is not needed anymore with updated calculation of AGI
# calculate Winf and W for each species
#Winf = np.asarray(LWa)*(np.asarray(Linf)**np.asarray(LWb))
#W    = (1./3.)*Winf
#print('Winf:',Winf)
#print('AGI_crit:',AGI_crit)
#print('o2thresh:',o2thresh)
#print('TPref:',TPref)

#--------
# NOTE: I don't think the order provided in LWa,LWb & Linf matches the order in which AGIcrit etc are loaded here!
# careful when indexing
# -> see above for finding the indices for the depth range!
#--------


In [7]:
#-----
# calculate AGI based on annual fields
# part I: load pO2 & t_insitu fields
#-----

which_sim = 'simAssp585' # for present-day, keep this as simAssp585! 
path1 = '/pscratch/sd/c/cnissen/files_toothfish_project_AGI/'+which_sim+'_reduced/'

#----
# PRESENT-DAY
#----

year_list1 = np.arange(1995,2014+1,1)

for yy in range(0,len(year_list1)):
    year1 = year_list1[yy]
    print('Load year ',year1)
    
    # pO2 data (mbar)
    file_pO2  = path1+'/pO2_fesom_'+which_sim+'_'+str(year1)+'0101_v2.nc'
    data_pO2   = xr.open_dataset(file_pO2)
    a1 = data_pO2['pO2'].values
    a1 = np.ma.masked_where(a1<0,a1)
    print('Min/Max pO2:',np.nanmin(a1),np.nanmax(a1))
    if yy==0:
        pO2_1      = a1 #data_pO2['pO2'].values
    else:
        pO2_1      = pO2_1+ a1 #data_pO2['pO2'].values
    data_pO2.close()
    del a1

    # T insitu (deg C)
    file_t_insitu   = path1+'/t_insitu_fesom_'+which_sim+'_'+str(year1)+'0101_v2.nc'
    data_t_insitu   = xr.open_dataset(file_t_insitu)
    a1 = data_t_insitu['t_insitu'].values
    a1 = np.ma.masked_where(a1<-3.0,a1)
    print('Min/Max t_insitu:',np.nanmin(a1),np.nanmax(a1))
    #print('Min/Max t_insitu:',np.min(data_t_insitu['t_insitu'].values),np.max(data_t_insitu['t_insitu'].values))
    if yy==0:
        t_insitu_1        = a1 #data_t_insitu['t_insitu'].values
    else:
        t_insitu_1        = t_insitu_1+ a1 #data_t_insitu['t_insitu'].values
    data_t_insitu.close()
    del a1
    
# calculate climatology
t_insitu_1 = t_insitu_1/len(year_list1)
pO2_1      = pO2_1/len(year_list1)
print('')
print('Min/Max t_insitu:',np.nanmin(t_insitu_1),np.nanmax(t_insitu_1))
print('Min/Max pO2:',np.nanmin(pO2_1),np.nanmax(pO2_1))
    
print('done')


Load year  1995
Min/Max pO2: 4.207563 225.77884
Min/Max t_insitu: -2.571443 15.545197
Load year  1996
Min/Max pO2: 2.8013957 225.59848
Min/Max t_insitu: -2.575366 14.551913
Load year  1997
Min/Max pO2: 1.8041228 225.43907
Min/Max t_insitu: -2.5624118 15.000496
Load year  1998
Min/Max pO2: 0.027826557 225.30064
Min/Max t_insitu: -2.5636892 15.028485
Load year  1999
Min/Max pO2: 1.8957909 225.04358
Min/Max t_insitu: -2.5640612 14.615901
Load year  2000
Min/Max pO2: 9.454568 224.61386
Min/Max t_insitu: -2.5593815 15.245595
Load year  2001
Min/Max pO2: 3.726922 224.15968
Min/Max t_insitu: -2.5575552 15.217681
Load year  2002
Min/Max pO2: 2.2234223 223.76306
Min/Max t_insitu: -2.5549266 14.752597
Load year  2003
Min/Max pO2: 2.194433 224.76065
Min/Max t_insitu: -2.5629275 15.570725
Load year  2004
Min/Max pO2: 2.6431694 229.60274
Min/Max t_insitu: -2.5545547 16.359056
Load year  2005
Min/Max pO2: 1.7453376 229.03206
Min/Max t_insitu: -2.5574455 14.253276
Load year  2006
Min/Max pO2: 3.34010

In [36]:
#-----
# calculate AGI based on annual fields
# part I: load pO2 & t_insitu fields
#-----

# the drift-corrected files are in folders called ssp585_reduced, ssp370_reduced etc. (simAssp585 has the non-drift-corrected files)
which_sim   = 'ssp585' 
which_drift = '2081_2100' # 2081_2100, 2091_2100, 2098_2100, 2090_2099

# example file name for drift-corrected files: t_insitu_fesom_ssp245_20910101_monthly_drift_corrected_2090_2099_minus_1995_2014_v2.nc
drift_corr ='monthly_drift_corrected_'+which_drift+'_minus_1995_2014_v2' 

path1 = '/pscratch/sd/c/cnissen/files_toothfish_project_AGI/'+which_sim+'_reduced/'

#which_sim = 'simAssp585'
#path1 = '/pscratch/sd/c/cnissen/files_toothfish_project_AGI/'+which_sim+'_reduced/'

#----
# FUTURE
#----

year_list2 = np.arange(int(which_drift[0:4]),int(which_drift[5:])+1,1)

for yy in range(0,len(year_list2)):
    year2 = year_list2[yy]
    print('Load year ',year2)
    
    # pO2 data (mbar)
    file_pO2  = path1+'/pO2_fesom_'+which_sim+'_'+str(year2)+'0101_'+drift_corr+'.nc'
    data_pO2   = xr.open_dataset(file_pO2)
    if yy==0:
        pO2_2      = data_pO2['pO2'].values
    else:
        pO2_2      = pO2_2+data_pO2['pO2'].values
    data_pO2.close()

    # T insitu (deg C)
    file_t_insitu   = path1+'/t_insitu_fesom_'+which_sim+'_'+str(year2)+'0101_'+drift_corr+'.nc'
    data_t_insitu   = xr.open_dataset(file_t_insitu)
    if yy==0:
        t_insitu_2        = data_t_insitu['t_insitu'].values
    else:
        t_insitu_2        = t_insitu_2+data_t_insitu['t_insitu'].values
    data_t_insitu.close()

# make sure future fields are handled the same way as historical fields!
pO2_2 = np.ma.masked_where(pO2_2<0,pO2_2)
t_insitu_2 = np.ma.masked_where(t_insitu_2<-3.0,t_insitu_2)

pO2_2[np.isnan(pO2_1)] = np.nan
t_insitu_2[np.isnan(t_insitu_1)] = np.nan

# calculate climatology
t_insitu_2 = t_insitu_2/len(year_list2)
pO2_2      = pO2_2/len(year_list2)
    
# volume data 
data_vol   = xr.open_dataset(file_mesh)
vol        = data_vol['volume']
data_vol.close()

print(vol.shape)
print(t_insitu_2.shape)
print(pO2_2.shape)


Load year  2081
Load year  2082
Load year  2083
Load year  2084
Load year  2085
Load year  2086
Load year  2087
Load year  2088
Load year  2089
Load year  2090
Load year  2091
Load year  2092
Load year  2093
Load year  2094
Load year  2095
Load year  2096
Load year  2097
Load year  2098
Load year  2099
Load year  2100
(88, 360, 1440)
(88, 360, 1440)
(88, 360, 1440)


In [37]:
#------
# weighted quantile function
# from here: https://stackoverflow.com/questions/21844024/weighted-percentile-using-numpy
#------
########
# to be double-check carefully! 
########

def weighted_quantile(values, quantiles, sample_weight=None, 
                      values_sorted=False, old_style=False):
    """ Very close to numpy.percentile, but supports weights.
    NOTE: quantiles should be in [0, 1]!
    :param values: numpy.array with data
    :param quantiles: array-like with many quantiles needed
    :param sample_weight: array-like of the same length as `array`
    :param values_sorted: bool, if True, then will avoid sorting of
        initial array
    :param old_style: if True, will correct output to be consistent
        with numpy.percentile.
    :return: numpy.array with computed quantiles.
    """
    values = np.array(values)
    quantiles = np.array(quantiles)
    if sample_weight is None:
        sample_weight = np.ones(len(values))
    sample_weight = np.array(sample_weight)
    assert np.all(quantiles >= 0) and np.all(quantiles <= 1), \
        'quantiles should be in [0, 1]'

    if not values_sorted:
        sorter = np.argsort(values)
        values = values[sorter]
        sample_weight = sample_weight[sorter]

    weighted_quantiles = np.cumsum(sample_weight) - 0.5 * sample_weight
    if old_style:
        # To be convenient with numpy.percentile
        weighted_quantiles -= weighted_quantiles[0]
        weighted_quantiles /= weighted_quantiles[-1]
    else:
        weighted_quantiles /= np.sum(sample_weight)
    return np.interp(quantiles, weighted_quantiles, values)

In [38]:
#----
# functions
#----

def get_habitable_volume(i,hab_field,vol,pO2,t_insitu,overlap,\
                                    species_list,species_names,TPref,o2thresh,AGI_crit,j_diff):
    
   # ### Get the in-habitat data for each species over the relevant depth range ###
   # file_habitat = habitat_file_names[i]
   # data_hab     = xr.open_dataset(file_habitat) # 2D habitat presence/absence data (1 for present or 0 for absent)
   # # make sure the habitat field has dimensions depth x lat x lon before applying it to the respective fields
   # num_depth = len(np.arange(index_min_depth[i],index_max_depth[i]+1,1)) 
   # hab_field = np.tile(data_hab['presence'].values,(num_depth,1,1))
   # data_hab.close()
    
    ### select the data/volume over the relevant depth range, correct with habitat array
    # NOTE: index_min_depth[i]:index_max_depth[i]    or.    index_min_depth[i]:index_max_depth[i]+1   ?
    vol_inhabitat = (vol.values[overlap,:,:]*hab_field) 
    dataout1      = (pO2[overlap,:,:]*hab_field)
    dataout2      = (t_insitu[overlap,:,:]*hab_field)
    
    #vol_inhabitat = (vol.values[index_min_depth[i]:index_max_depth[i]+1,:,:]*hab_field) 
    #dataout1      = (pO2[index_min_depth[i]:index_max_depth[i]+1,:,:]*hab_field)
    #dataout2      = (t_insitu[index_min_depth[i]:index_max_depth[i]+1,:,:]*hab_field)
    
    #print(dataout1.shape) # depth x lat x lon

    # use all months
    dataout1b = dataout1[:,:,:].ravel() # pO2
    dataout2b = dataout2[:,:,:].ravel() # temp
    # make sure volume array has dimensions months x depths x lat x lon before applying ravel()
    vol_inhabitat_b = np.copy(vol_inhabitat).ravel() #np.tile(vol_inhabitat,(12,1,1,1)).ravel() 
    
    # make sure to only keep nodes that are available for current species
    ind = np.where((~np.isnan(dataout1b)))[0] # every available node
    dataout1b = dataout1b[ind]
    dataout2b = dataout2b[ind]
    vol_inhabitat_b = vol_inhabitat_b[ind]
    ind = np.where((dataout1b!=0))[0] # every available node
    dataout1b = dataout1b[ind] # pO2
    dataout2b = dataout2b[ind] # temp
    vol_inhabitat_b = vol_inhabitat_b[ind]
    del ind
    
  #  print('Min/Max pO2 in habitat:',np.min(dataout1b),np.max(dataout1b))
  #  print('Min/Max t insitu in habitat:',np.min(dataout2b),np.max(dataout2b))
    
    totalvol_inhabitat = np.nansum(vol_inhabitat_b) # total in-habitat volume (accounting for bathymetry)
  #  print('totalvol_inhabitat (3D):',totalvol_inhabitat) 
    vol_weights = vol_inhabitat_b/np.sum(vol_inhabitat_b)
    #print(np.sum(vol_weights)) # as a check, should be 1
    
    # order of species in W, Winf etc not the same as in this i-loop!
    # correct for that here (same was done for index_min_depth and index_max_depth)
    iii = species_list.index(species_names[i])
    
    #------
    # get Tpref & O2thresh -> see plot_AGI_toothfish_TPref_O2thresh_save_as_netcdf.ipynb
    #   Tpref: 50th percentile in habitat (volume weighted)
    #   O2thresh: 10th percentile in habitat (volume weighted)
    tpref_species    = TPref[i]
    o2thresh_species = o2thresh[i]   
    #------
    
    #----
    # calculate AGIcrit: 10th percentile in habitat (volume wieghted)
    
    #w_ratio = (W[iii]**(1-d))/(Winf[iii]**(1-d))
    # UPDATE Jan 2023: adapt calculation of AGI to what is in Anne's paper (W, W_inf etc not needed anymore)
    w_ratio = ((1./3.)**(1-d))
    a1 = np.exp((j_diff/(tpref_species+273.15)) - (j_diff/(dataout2b+273.15)))
    
    AGI = dataout1b/(o2thresh_species*w_ratio*a1)
    #print('Min/Max AGI in 3D habitat:',np.min(AGI),np.max(AGI))
    
    #print(AGI.shape)
    #print(vol_inhabitat_b.shape)
    
    res = weighted_quantile(AGI, [0.1], sample_weight=vol_weights)
    
    ind_hab = np.where(AGI>AGI_crit[i])[0]
    
  #  #---
  #  # correction November 2023
  #  mask_habitat = np.zeros_like(AGI) 
  #  AGI[AGI.mask==True]=-999 # DOES THIS line change the problem with identifying prey at a depth level where it shouldn't be?
  #  ind_hab = np.where(AGI>AGI_crit[i]) # if AGI contains masked cells, these will be interpreted as >1 here!!!!
  #  mask_habitat[ind_hab] = 1
  #  mask_habitat[mask_habitat.mask==True]=0
  #  #---   
  #  
  #  # old code
  #  #ind_hab = np.where(AGI>AGI_crit[i]) #[0]
  #  #mask_habitat = np.zeros_like(AGI) #
  #  #mask_habitat[ind_hab] = 1
    
    
    #print(ind_hab.shape)
    #print('volume above AGIcrit:',np.sum(vol_inhabitat_b[ind_hab]))
    #print('total volume:',totalvol_inhabitat)
    #print('Ratio:',np.sum(vol_inhabitat_b[ind_hab])/totalvol_inhabitat)
    
    # return: AGIcrit_future, volume above AGIcrit_present_day, total volume
    return res[0],np.sum(vol_inhabitat_b[ind_hab]),np.sum(vol_inhabitat_b)
    
def get_mask_viable_habitat(i,hab_field,vol,pO2,t_insitu,index_min_depth,index_max_depth,\
                                    species_list,species_names,TPref,o2thresh,AGI_crit,j_diff,mask):
    # NOTE: "mask" is 2D field; can be an MPA or the whole Southern Ocean (array of ones in this case)
    
    # turn mask into 3D
    mask3D = np.tile(mask,[88,1,1])
    #mask3D = mask3D[index_min_depth[i]:index_max_depth[i]+1,:,:] # select depth levels of interest
    
    # set mask outside of depth range to zero
    if index_min_depth[i]>0:
        mask3D[0:index_min_depth[i]:,:] = 0
    mask3D[index_max_depth[i]:,:,:] = 0
    
    ### select the data/volume over the relevant depth range, correct with habitat array
    # NOTE: index_min_depth[i]:index_max_depth[i]    or.    index_min_depth[i]:index_max_depth[i]+1   ?
    vol_inhabitat = (vol.values[:,:,:]*hab_field*mask3D) 
    dataout1      = (pO2[:,:,:]*hab_field*mask3D)
    dataout2      = (t_insitu[:,:,:]*hab_field*mask3D)
    #print(dataout1.shape) # depth x lat x lon
    
    # use all months -> don't "ravel" here to keep the depths x lat x lon format
    dataout1b = dataout1[:,:,:] #.ravel() # pO2
    dataout2b = dataout2[:,:,:] #.ravel() # temp
    # make sure volume array has dimensions months x depths x lat x lon before applying ravel()
    vol_inhabitat_b = np.copy(vol_inhabitat) #.ravel() #np.tile(vol_inhabitat,(12,1,1,1)).ravel() 
    
    dataout2b       = np.ma.masked_where(np.isnan(dataout1b),dataout2b) # every available node
    vol_inhabitat_b = np.ma.masked_where(np.isnan(dataout1b),vol_inhabitat_b)
    dataout1b       = np.ma.masked_where(np.isnan(dataout1b),dataout1b)
    
    dataout2b       = np.ma.masked_where(dataout1b==0,dataout2b) # every available node
    vol_inhabitat_b = np.ma.masked_where(dataout1b==0,vol_inhabitat_b)
    dataout1b       = np.ma.masked_where(dataout1b==0,dataout1b)
    
    vol_weights = vol_inhabitat_b/np.nansum(vol_inhabitat_b)
    
    # order of species in W, Winf etc not the same as in this i-loop!
    # correct for that here (same was done for index_min_depth and index_max_depth)
    iii = species_list.index(species_names[i])
    
    #------
    # get Tpref & O2thresh -> see plot_AGI_toothfish_TPref_O2thresh_save_as_netcdf.ipynb
    #   Tpref: 50th percentile in habitat (volume weighted)
    #   O2thresh: 10th percentile in habitat (volume weighted)
    tpref_species    = TPref[i]
    o2thresh_species = o2thresh[i]   
    #------
    
    #----
    # calculate AGIcrit: 10th percentile in habitat (volume wieghted)
    
    #w_ratio = (W[iii]**(1-d))/(Winf[iii]**(1-d))
    # UPDATE Jan 2023: adapt calculation of AGI to what is in Anne's paper (W, W_inf etc not needed anymore)
    w_ratio = ((1./3.)**(1-d))
    a1 = np.exp((j_diff/(tpref_species+273.15)) - (j_diff/(dataout2b+273.15)))
    
    AGI = dataout1b/(o2thresh_species*w_ratio*a1)
    
    #res = weighted_quantile(AGI, [0.1], sample_weight=vol_weights)
    
    ind_hab = np.where(AGI>AGI_crit[i]) #[0]
    
    mask_habitat = np.zeros_like(AGI) #
    mask_habitat[ind_hab] = 1

    return mask_habitat,vol_inhabitat_b,np.sum(vol_inhabitat_b[ind_hab])
    

In [39]:
#---
# get habitat loss for each species and each depth intervall
#---

from netCDF4 import Dataset
from datetime import datetime
from tqdm import tqdm

today = datetime.today()

save_as_netcdf = True

# first version
#depth_list1 = [0,   0,  250,750, 1500]
#depth_list2 = [3500,250,750,1500,3500]

# second version (motivated by temp & O2 changes in toothfish habitat)
depth_list1 = [0,   0,  400,700, 1000]
depth_list2 = [3500,400,700,1000,3500]

levels[0]=0
    
habitat_loss = np.zeros([len(species_names),len(depth_list1)])
for i in range(0,len(species_names)):
    print(i,species_names[i])
    print(habitat_file_names[i])
    
    for dd in tqdm(range(0,len(depth_list1))):
        depth1,depth2 = depth_list1[dd],depth_list2[dd] #750,1500 #250,750 #0,250 #1000,2000 #0,3000
        ind_depths = np.where((levels>=depth1) & (levels<depth2))[0]

        ### Get the in-habitat data for each species over the relevant depth range ###
        file_habitat = habitat_file_names[i]
        data_hab     = xr.open_dataset(file_habitat) # 2D habitat presence/absence data (1 for present or 0 for absent)
        av_depths = np.arange(index_min_depth[i],index_max_depth[i]+1,1)
        
        overlap = np.array(list(set(ind_depths) & set(av_depths)))
        
        if len(overlap)>0: # only of some depth levels are available for current species
            # make sure the habitat field has dimensions depth x lat x lon before applying it to the respective fields
            num_depth = len(overlap) 
            hab_field = np.tile(data_hab['presence'].values,(num_depth,1,1))
            data_hab.close()

            AGI_crit_present,vol_hab1,tot_vol = get_habitable_volume(i,hab_field,vol,pO2_1,t_insitu_1,\
                                                                     overlap,\
                                            species_list,species_names,TPref,o2thresh,AGI_crit,j_diff)
            del tot_vol

            # get habitable volume in present-day 
            AGI_crit_future,vol_hab2,tot_vol = get_habitable_volume(i,hab_field,vol,pO2_2,t_insitu_2,\
                                                                    overlap,\
                                            species_list,species_names,TPref,o2thresh,AGI_crit,j_diff)

            #print('AGIcrit present-day vs future:',AGI_crit[i],AGI_crit_future)
            #print('habitable volume present-day (volume above present-day AGIcrit, =90%):',vol_hab1)
            #print('habitable volume future (volume above present-day AGIcrit):',vol_hab2)
            #print(tot_vol)
            #print('Ratio of future habitable volume to total volume',100*vol_hab2/tot_vol)
            #print('Habitat loss (90% minus future habitable volume):',90-100*vol_hab2/tot_vol)
            # in the present-day, the above is 90% (per definition)
            # anything below 90% is a habitat loss
            habitat_loss[i,dd] = 90-100*vol_hab2/tot_vol
            
            del AGI_crit_present,AGI_crit_future,vol_hab1,vol_hab2,tot_vol,hab_field

        del overlap,av_depths,data_hab,file_habitat,ind_depths,depth1,depth2
        
    #----
    # save as netcdf
    #----
    if save_as_netcdf:
        #-----
        # create file & define dimensions
        #-----
        savepath = '/pscratch/sd/c/cnissen/AGI_toothfish_project/habitat_loss/'
        netcdf_name = 'Habitat_loss_'+str(year_list2[0])+'_'+str(year_list2[-1])+'_rel_to_'+\
                            str(year_list1[0])+'_'+str(year_list1[-1])+'_'+which_sim+'_'+drift_corr+'.nc'
        if not os.path.exists(savepath+netcdf_name):
            print ('Create file '+savepath+netcdf_name)
            w_nc_fid = Dataset(savepath+netcdf_name, 'w', format='NETCDF4_CLASSIC')
            # create dimension & variable
            w_nc_fid.createDimension('species_names', len(species_names)) 
            w_nc_fid.createDimension('depth_intervals', len(depth_list1)) 
            w_nc_fid.close()

            #-----
            # global information
            #-----
            #----
            script_name = 'plot_PAPER_AGI_toothfish_present_vs_future_habitat_ALL_TOGETHER_save_as_netcdf.ipynb'
            w_nc_fid = Dataset(savepath+netcdf_name, 'r+', format='NETCDF4_CLASSIC')
            w_nc_fid.species_names1 = '0=Dissostichus mawsoni, 1=Trematomus loennbergii, 2=Macrourus whitsoni, 3=Galiteuthis glacialis'
            w_nc_fid.species_names2 = '4=Chionodraco hamatus, 5=Anotopterus vorax, 6=Chaenodraco wilsoni, 7=Kondakovia longimana'
            w_nc_fid.species_names3 = '8=Cryodraco antarcticus, 9=Muraenolepis microps, 10=Psychroteuthis glacialis, 11=Gymnodraco acuticeps'
            w_nc_fid.species_names4 = '12=Amblyraja georgiana, 13=Antarctomysis maxima, 14=Lepidonotothen 15=squamifrons, 16=Mesonychoteuthis hamiltoni'
            w_nc_fid.species_names5 = '17=Pleuragramma antarctica, 18=Euphausia superba, 19=Notothenia coriiceps, 20=Trematomus lepidorhinus'
            w_nc_fid.species_names6 = '21=Pagothenia borchgrevinki, 22=Lycenchelys aratrirostris, 23=Neopagetopsis ionah, 24=Chaenocephalus aceratus'
            w_nc_fid.species_names7 = '25=Trematomus hansoni, 26=Chionobathyscus dewitti, 27=Euphausia crystallorophias, 28=Notothenia rossii'
            w_nc_fid.species_names8 = '29=Bathyraja maccaini'
            w_nc_fid.AGI_crit = 'AGIcrit based on monthly mean climatology'
            w_nc_fid.present_day = str(year_list1[0])+'-'+str(year_list1[-1])
            w_nc_fid.future = str(year_list2[0])+'-'+str(year_list2[-1])
            w_nc_fid.which_scenario = which_sim
            w_nc_fid.history = "Created " + today.strftime("%d/%m/%y")
            w_nc_fid.script = script_name+" (for internal information only)"
            w_nc_fid.close()
            
            #-----
            # define variables
            #-----
            w_nc_fid = Dataset(savepath+netcdf_name, 'r+', format='NETCDF4_CLASSIC')      # Create and open new netcdf file to write to
            w_nc_var1a = w_nc_fid.createVariable('habitat_loss', 'f4',('species_names','depth_intervals'))
            w_nc_var1a.description = 'Loss of viable habitat (AGI>AGIcrit)'
            w_nc_var1a.units = 'Mio km3'

            w_nc_var1a = w_nc_fid.createVariable('depth_upper_bound', 'f4',('depth_intervals'))
            w_nc_var1a = w_nc_fid.createVariable('depth_lower_bound', 'f4',('depth_intervals'))

            w_nc_fid.close() 

        w_nc_fid = Dataset(savepath+netcdf_name, 'r+', format='NETCDF4_CLASSIC')      # Create and open new netcdf file to write to
        w_nc_fid.variables['depth_upper_bound'][:]= depth_list1
        w_nc_fid.variables['depth_lower_bound'][:]= depth_list2
        w_nc_fid.variables['habitat_loss'][i,:]= habitat_loss[i,:]
        w_nc_fid.close() 
    
print('done')
        

0 Dissostichus mawsoni
/pscratch/sd/c/cnissen/AGI_toothfish_project/all_prey_plus_toothfish/Reviewed_Dissostichus_mawsoni_modelgrid_boolean.nc


100%|██████████| 5/5 [00:09<00:00,  1.87s/it]


1 Trematomus loennbergii
/pscratch/sd/c/cnissen/AGI_toothfish_project/all_prey_plus_toothfish/Reviewed_Trematomus_loennbergii_modelgrid_boolean.nc


100%|██████████| 5/5 [00:04<00:00,  1.10it/s]


2 Macrourus whitsoni
/pscratch/sd/c/cnissen/AGI_toothfish_project/all_prey_plus_toothfish/Reviewed_Macrourus_whitsoni_modelgrid_boolean.nc


...

27 Notothenia rossii
/pscratch/sd/c/cnissen/AGI_toothfish_project/all_prey_plus_toothfish/Default_Notothenia_rossii_modelgrid_boolean.nc


100%|██████████| 5/5 [00:02<00:00,  1.91it/s]


28 Bathyraja maccaini
/pscratch/sd/c/cnissen/AGI_toothfish_project/all_prey_plus_toothfish/Reviewed_Bathyraja_maccaini_modelgrid_boolean.nc


100%|██████████| 5/5 [00:01<00:00,  2.81it/s]

done
